This one is to look at delta+, delta0, and their antis; these decay into ns, ps, and pions, and appeared to do ever so slightly better when tested on the classifier. Do they always have both ns and ps? try to isolate the two cases and test separately? Let's go exploring!

This notebook basically does all the dataprocessing for the other notebook, w12_effvsdeltar where I map the accuracy of testing the data on the classifier in bins of deltar

In [1]:
#imports
import uproot as ur
import awkward as ak
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "6" #specify GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import graph_util as gu
import plot_util as pu

from scipy.interpolate import interp1d

from Week7_PFNutils import *

2021-07-21 06:55:34.847858: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-21 06:55:49.022404: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-21 06:55:49.052771: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-07-21 06:55:49.292279: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Found device 0 with properties: 
pciBusID: 0000:3d:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-07-21 06:55:49.292340: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-21 06:55:49.358096: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2

In [2]:
model_path = '/fast_scratch/atlas_images/v01-45/' 
data_path = '/data/atlas/'

In [3]:
import vector as v

import atlas_mpl_style as ampl #makes matplotlib follow atlas formal style
ampl.use_atlas_style()

v.register_awkward()

In [86]:
events = ur.open(model_path+'delta_medium.root')

tree = events['EventTree']

In [5]:
branches = tree.arrays()

In [6]:
truthE = branches.cluster_hitsTruthE
truthEIndex = branches.cluster_hitsTruthIndex
truthID = branches.truthPartPdgId
trutheta = branches.truthPartEta
truthphi = branches.truthPartPhi
truthpt = branches.truthPartPt
truthmass = branches.truthPartMass
clusE = branches.cluster_E
clusEta = branches.cluster_Eta
clusPhi = branches.cluster_Phi
clusPt = branches.cluster_Pt

In [7]:
ID_delp0 = []
phi_delp0 = []
eta_delp0 = []
pt_delp0 = []
m_delp0 = []

clusE_delp0 = []
clusEta_delp0 = []
clusPhi_delp0 = []
clusPt_delp0 = []

index = []

for i in range(len(truthID)):
    if (2214 in truthID[i]) | (-2214 in truthID[i]) | (2114 in truthID[i]) | (-2114 in truthID[i]):
        ID_delp0.append(truthID[i])
        phi_delp0.append(truthphi[i])
        eta_delp0.append(trutheta[i])
        pt_delp0.append(truthpt[i])
        m_delp0.append(truthmass[i])
        clusE_delp0.append(clusE[i])
        clusEta_delp0.append(clusEta[i])
        clusPhi_delp0.append(clusPhi[i])
        clusPt_delp0.append(clusPt[i])
        
        index.append(i)

In [10]:
#well okay lets start without the antis
pnon = []
nnop = []
for i in range(len(ID_delp0)):
    if (2212 in ID_delp0[i]) & ~(2112 in ID_delp0[i]):
        pnon.append(i)
    elif ~(2212 in ID_delp0[i]) & (2112 in ID_delp0[i]):
        nnop.append(i)

In [14]:
nnop, pnon #okay so there are plenty with one and not the other

([0,
  2,
  23,
  30,
  32,
  34,
  37,
  46,
  47,
  52,
  54,
  60,
  63,
  66,
  67,
  74,
  75,
  79,
  84,
  88,
  98,
  99,
  105,
  108,
  111,
  112,
  114,
  115,
  121,
  130,
  132,
  133,
  141,
  144,
  149,
  156,
  165,
  172,
  185,
  186,
  187,
  188,
  190,
  192,
  193,
  195,
  197,
  199,
  203,
  205,
  212,
  219,
  220,
  221,
  226,
  229,
  230,
  232,
  233,
  237,
  238,
  240,
  243,
  246,
  247,
  250,
  256,
  261,
  264,
  270,
  272,
  274,
  277,
  283,
  285,
  293,
  294,
  298,
  300,
  303,
  304,
  309,
  320,
  333,
  334,
  338,
  344,
  345,
  348,
  349,
  356,
  357,
  366,
  372,
  377,
  381,
  382,
  390,
  391,
  396,
  399,
  403,
  416,
  425,
  427,
  429,
  430,
  432,
  434,
  435,
  436,
  437,
  439,
  441,
  446,
  459,
  467,
  470,
  478,
  480,
  491,
  495,
  508,
  513,
  517,
  519,
  520,
  522,
  532,
  542,
  545,
  546,
  549,
  551,
  552,
  556,
  558,
  560,
  562,
  570,
  584,
  585,
  588,
  594,
  600,
  607,
  

In [18]:
[ID_delp0[i] for i in nnop]

[<Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2214, 2112, 211, 22] type='4 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2114, 2112, 22] type='3 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [2214, 2112, 211, 22] type='4 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2214, 2112, 211, 22] type='4 * int32'>,
 <Array [2114, 2112, 111, 22, 22] type='5 * int32'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [2214, 21

In [21]:
ID_nnop = np.array(ak.flatten([ID_delp0[x] for x in nnop]))
phi_nnop = np.array(ak.flatten([phi_delp0[x] for x in nnop]))
eta_nnop = np.array(ak.flatten([eta_delp0[x] for x in nnop]))
pt_nnop = np.array(ak.flatten([pt_delp0[x] for x in nnop]))
m_nnop = np.array(ak.flatten([m_delp0[x] for x in nnop]))
clusE_nnop = np.array(ak.flatten([clusE_delp0[x] for x in nnop]))
clusEta_nnop = np.array(ak.flatten([clusEta_delp0[x] for x in nnop]))
clusPhi_nnop = np.array(ak.flatten([clusPhi_delp0[x] for x in nnop]))
clusPt_nnop = np.array(ak.flatten([clusPt_delp0[x] for x in nnop]))

In [22]:
ID_pnon = np.array(ak.flatten([ID_delp0[x] for x in pnon]))
phi_pnon = np.array(ak.flatten([phi_delp0[x] for x in pnon]))
eta_pnon = np.array(ak.flatten([eta_delp0[x] for x in pnon]))
pt_pnon = np.array(ak.flatten([pt_delp0[x] for x in pnon]))
m_pnon = np.array(ak.flatten([m_delp0[x] for x in pnon]))
clusE_pnon = np.array(ak.flatten([clusE_delp0[x] for x in pnon]))
clusEta_pnon = np.array(ak.flatten([clusEta_delp0[x] for x in pnon]))
clusPhi_pnon = np.array(ak.flatten([clusPhi_delp0[x] for x in pnon]))
clusPt_pnon = np.array(ak.flatten([clusPt_delp0[x] for x in pnon]))

I think I need to flatten them and then make the flag out of that, they should all contain n or p so shouldn't be hard?
how do I mask now??? 

In the other one I had a cut for all the pi0 and a cut for all the protons... so that ensured they were both there. but not the other particles? I guess they weren't included? Do I want to do that for this???

In [23]:
len(ID_nnop), len(ID_pnon)

(108754, 110918)

In [57]:
#so all I need is the mask, and I'll do two: one with delta+ -> n and pi+, and one with delta0 -> p and pi0

#so first its JUST DELTA+!!!!!!! note! warning!!
pnonind = []
nnopind = []    

for i in range(len(truthID)): #basically: delta+, two categories: pip & n (without any ps) vs pi0 & p (no ns)
    if (2214 in truthID[i]) & ~(2212 in truthID[i]) & (2112 in truthID[i]) & (211 in truthID[i]):
        nnopind.append(i)
    elif (2214 in truthID[i]) & (2212 in truthID[i]) & ~(2112 in truthID[i]) & (111 in truthID[i]):
        pnonind.append(i)
        
#this is literally still so janky but whatever
flag = np.zeros(len(truthID))
flag[pnonind] = 1
flag[nnopind] = 2 #just doing pi+ and n to start

In [58]:
flag

array([0., 0., 0., ..., 0., 0., 0.])

In [59]:
maskpnon = flag == 1
masknnop = flag == 2

In [60]:
import convert_sets_splitdelta as cs

In [61]:
cs.doit(masknnop, data_path+'w12_nnop', 211, 2112) #okayyy now we're getting somewhere

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
30788 30788 30788
Normalizing
Writing out
Done! /data/atlas/w12_nnop


In [63]:
cs.doit(maskpnon, data_path+'w12_pnon', 111, 2212)

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
55001 55001 55001
Normalizing
Writing out
Done! /data/atlas/w12_pnon


In [64]:
filennop = np.load(data_path+'w12_nnop.npz')
filepnon = np.load(data_path+'w12_pnon.npz')

In [65]:
Xnnop = filennop['X']
clus_etannop = filennop['clus_eta']
clus_ptnnop = filennop['clus_pt']
clus_Ennop = filennop['clus_e']
clus_etnnop = filennop['clus_e_t']
deltarnnop = filennop['deltaR']

print(len(Xnnop))

27078


In [66]:
Xpnon = filepnon['X']
clus_etapnon = filepnon['clus_eta']
clus_ptpnon = filepnon['clus_pt']
clus_Epnon = filepnon['clus_e']
clus_etpnon = filepnon['clus_e_t']
deltarpnon = filepnon['deltaR']

print(len(Xpnon))

49616


In [71]:
def doit(dat, outfile, size):
    X, clus_eta, clus_pt, clus_E, clus_et, deltar = dat
    X_all = np.array(X[:size])
    eta_all = np.array(clus_eta[:size])
    pt_all = np.array(clus_pt[:size])
    E_all = np.array(clus_E[:size])
    et_all = np.array(clus_et[:size])
    deltar_all = np.array(deltar[:size])
    
    (X_train, X_val, X_test,  
     eta_train, eta_val, eta_test, 
     ET_train, ET_val, ET_test, 
     pt_train, pt_val, pt_test, 
     Eng_train, Eng_val, Eng_test,
     deltar_train, deltar_val, deltar_test) = data_split(X_all, eta_all, et_all, pt_all, E_all, deltar_all, val=100, test=int(size/2.))
    
    np.savez(data_path+outfile, X_train, X_val, X_test, eta_train, eta_val, eta_test, ET_train, ET_val, ET_test, pt_train, pt_val, pt_test, Eng_train, Eng_val, Eng_test, deltar_train, deltar_val, deltar_test)

In [72]:
doit((Xpnon, clus_etapnon, clus_ptpnon, clus_Epnon, clus_etpnon, deltarpnon), 'nn_pnon_w12.npz', 49000)

In [73]:
doit((Xnnop, clus_etannop, clus_ptnnop, clus_Ennop, clus_etnnop, deltarnnop), 'nn_nnop_w12.npz', 27000)

In [78]:
#from here, doing the same for delta0:

pnonind = []
nnopind = []    

for i in range(len(truthID)): #basically: delta+, two categories: pip & n (without any ps) vs pi0 & p (no ns)
    if (2114 in truthID[i]) & ~(2112 in truthID[i]) & (2212 in truthID[i]) & (-211 in truthID[i]):
        nnopind.append(i)
    elif (2114 in truthID[i]) & (2112 in truthID[i]) & ~(2212 in truthID[i]) & (111 in truthID[i]):
        pnonind.append(i)
        
#this is literally still so janky but whatever
flag = np.zeros(len(truthID))
flag[pnonind] = 1
flag[nnopind] = 2 #just doing pi+ and n to start

In [79]:
maskpnon = flag == 1
masknnop = flag == 2

In [80]:
cs.doit(masknnop, data_path+'w12_del0_nnop', -211, 2212) #okay this is the right order, switched from delta+
cs.doit(maskpnon, data_path+'w12_del0_pnon', 111, 2112)

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
31172 31172 31172
Normalizing


/home/kaelaekd/start_tf/LCStudies/classifier/convert_sets_splitdelta.py:73: RuntimeWarning: divide by zero encountered in log
  cell_e = np.nan_to_num(np.log(cell_e), posinf = 0, neginf=0)


Writing out
Done! /data/atlas/w12_del0_nnop
Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
55218 55218 55218
Normalizing
Writing out
Done! /data/atlas/w12_del0_pnon


In [81]:
filennop = np.load(data_path+'w12_del0_nnop.npz')
filepnon = np.load(data_path+'w12_del0_pnon.npz')

In [82]:
Xnnop = filennop['X']
clus_etannop = filennop['clus_eta']
clus_ptnnop = filennop['clus_pt']
clus_Ennop = filennop['clus_e']
clus_etnnop = filennop['clus_e_t']
deltarnnop = filennop['deltaR']

print(len(Xnnop))

27567


In [83]:
Xpnon = filepnon['X']
clus_etapnon = filepnon['clus_eta']
clus_ptpnon = filepnon['clus_pt']
clus_Epnon = filepnon['clus_e']
clus_etpnon = filepnon['clus_e_t']
deltarpnon = filepnon['deltaR']

print(len(Xpnon))

49843


In [84]:
doit((Xpnon, clus_etapnon, clus_ptpnon, clus_Epnon, clus_etpnon, deltarpnon), 'nn_del0pnon_w12.npz', 49000)

In [85]:
doit((Xnnop, clus_etannop, clus_ptnnop, clus_Ennop, clus_etnnop, deltarnnop), 'nn_del0nnop_w12.npz', 27000)

In [87]:
#from here, doing the same for anti-delta0: (sorry I'm overwriting variable names whoopsie)

pnonind = []
nnopind = []    

for i in range(len(truthID)): #basically: delta+, two categories: pip & n (without any ps) vs pi0 & p (no ns)
    if (-2114 in truthID[i]) & ~(-2112 in truthID[i]) & (-2212 in truthID[i]) & (211 in truthID[i]):
        nnopind.append(i)
    elif (-2114 in truthID[i]) & (-2112 in truthID[i]) & ~(-2212 in truthID[i]) & (111 in truthID[i]):
        pnonind.append(i)
        
#this is literally still so janky but whatever
flag = np.zeros(len(truthID))
flag[pnonind] = 1
flag[nnopind] = 2 #just doing pi+ and n to start

In [88]:
maskpnon = flag == 1
masknnop = flag == 2

In [89]:
cs.doit(masknnop, data_path+'w12_antidel0_nnop', 211, -2212) #omg dope it works!
cs.doit(maskpnon, data_path+'w12_antidel0_pnon', 111, -2112)

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
32214 32214 32214
Normalizing
Writing out
Done! /data/atlas/w12_antidel0_nnop
Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
59217 59217 59217
Normalizing
Writing out
Done! /data/atlas/w12_antidel0_pnon


In [90]:
filennop = np.load(data_path+'w12_antidel0_nnop.npz')
filepnon = np.load(data_path+'w12_antidel0_pnon.npz')

In [91]:
Xnnop = filennop['X']
clus_etannop = filennop['clus_eta']
clus_ptnnop = filennop['clus_pt']
clus_Ennop = filennop['clus_e']
clus_etnnop = filennop['clus_e_t']
deltarnnop = filennop['deltaR']

print(len(Xnnop))

28328


In [92]:
Xpnon = filepnon['X']
clus_etapnon = filepnon['clus_eta']
clus_ptpnon = filepnon['clus_pt']
clus_Epnon = filepnon['clus_e']
clus_etpnon = filepnon['clus_e_t']
deltarpnon = filepnon['deltaR']

print(len(Xpnon))

53425


In [94]:
doit((Xpnon, clus_etapnon, clus_ptpnon, clus_Epnon, clus_etpnon, deltarpnon), 'nn_antidel0pnon_w12.npz', 53000)

In [93]:
doit((Xnnop, clus_etannop, clus_ptnnop, clus_Ennop, clus_etnnop, deltarnnop), 'nn_antidel0nnop_w12.npz', 28000)

In [95]:
#and here for anti-delta+

pnonind = []
nnopind = []    

for i in range(len(truthID)): #basically: delta+, two categories: pip & n (without any ps) vs pi0 & p (no ns)
    if (-2214 in truthID[i]) & ~(-2212 in truthID[i]) & (-2112 in truthID[i]) & (-211 in truthID[i]):
        nnopind.append(i)
    elif (-2214 in truthID[i]) & (-2212 in truthID[i]) & ~(-2112 in truthID[i]) & (111 in truthID[i]):
        pnonind.append(i)
        
#this is literally still so janky but whatever
flag = np.zeros(len(truthID))
flag[pnonind] = 1
flag[nnopind] = 2 #just doing pi+ and n to start

In [96]:
maskpnon = flag == 1
masknnop = flag == 2

In [97]:
cs.doit(masknnop, data_path+'w12_antidel+_nnop', -211, -2112) #omg dope it works!
cs.doit(maskpnon, data_path+'w12_antidel+_pnon', 111, -2212)

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
32841 32841 32841
Normalizing
Writing out
Done! /data/atlas/w12_antidel+_nnop
Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
57912 57912 57912
Normalizing
Writing out
Done! /data/atlas/w12_antidel+_pnon


In [98]:
filennop = np.load(data_path+'w12_antidel+_nnop.npz')
filepnon = np.load(data_path+'w12_antidel+_pnon.npz')

In [99]:
Xnnop = filennop['X']
clus_etannop = filennop['clus_eta']
clus_ptnnop = filennop['clus_pt']
clus_Ennop = filennop['clus_e']
clus_etnnop = filennop['clus_e_t']
deltarnnop = filennop['deltaR']

print(len(Xnnop))

28937


In [100]:
Xpnon = filepnon['X']
clus_etapnon = filepnon['clus_eta']
clus_ptpnon = filepnon['clus_pt']
clus_Epnon = filepnon['clus_e']
clus_etpnon = filepnon['clus_e_t']
deltarpnon = filepnon['deltaR']

print(len(Xpnon))

52245


In [101]:
doit((Xpnon, clus_etapnon, clus_ptpnon, clus_Epnon, clus_etpnon, deltarpnon), 'nn_antidel+pnon_w12.npz', 52000)
doit((Xnnop, clus_etannop, clus_ptnnop, clus_Ennop, clus_etnnop, deltarnnop), 'nn_antidel+nnop_w12.npz', 28000)